In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pandas_datareader.data as web
from pandas.api.types import CategoricalDtype
import os
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

import sys, os
sys.path.append(os.path.abspath(".."))

from pantheon.config import RAW_DATA_DIR, TICKERS

2025-10-06 19:54:57.424 | INFO     | pantheon.config:<module>:11 - PROJ_ROOT path is: C:\Users\User\Documents\GitHub\pantheon-quant


In [10]:
output_dir = "fund_rawdata"
os.makedirs(RAW_DATA_DIR / output_dir, exist_ok=True)

URLS = {"BBAS3": "https://www.oceans14.com.br/acoes/banco-do-brasil/bbas3/balanco-dividendos",
        "ITUB4": "https://www.oceans14.com.br/acoes/itau-unibanco/itub4/balanco-dividendos", "BBDC4": "https://www.oceans14.com.br/acoes/bradesco/bbdc4/balanco-dividendos", 
        "B3SA3": "https://www.oceans14.com.br/acoes/b3/b3sa3/balanco-dividendos", 
        "BPAC11": "https://www.oceans14.com.br/acoes/banco-btg-pactual/bpac11/balanco-dividendos",
        "PETR4": "https://www.oceans14.com.br/acoes/petrobras/petr4/balanco-dividendos", 
        "PRIO3": "https://www.oceans14.com.br/acoes/prio3/balanco-dividendos", 
        "ELET3": "https://www.oceans14.com.br/acoes/eletrobras/elet3/balanco-dividendos", 
        "ENEV3": "https://www.oceans14.com.br/acoes/eneva/enev3/balanco-dividendos", 
        "EQTL3": "https://www.oceans14.com.br/acoes/equatorial/eqtl3/balanco-dividendos",
        "VALE3": "https://www.oceans14.com.br/acoes/vale/vale3/balanco-dividendos", 
        "ABEV3": "https://www.oceans14.com.br/acoes/abev/abev3/balanco-dividendos", 
        "EMBR3": "https://www.oceans14.com.br/acoes/embrapa/embr3/balanco-dividendos", 
        "CSAN3": "https://www.oceans14.com.br/acoes/cosan/csan3/balanco-dividendos", 
        "WEGE3": "https://www.oceans14.com.br/acoes/weg/wege3/balanco-dividendos"
}

email = 'gv.freitas2003@usp.br'
password = 'temp-quantamental'

loginp = 'https://www.oceans14.com.br/usuarios/frmLogin.aspx'
driver = webdriver.Firefox()
driver.get(loginp)
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, "#txtLogin").send_keys(email)
driver.find_element(By.CSS_SELECTOR, "#txtSenha").send_keys(password)
time.sleep(2)
buttom = driver.find_element(By.CSS_SELECTOR, "#cmdLogar")
webdriver.ActionChains(driver).click(buttom).perform()

#getting correct urls for each ticker
for ticker in TICKERS:
    
    ticker = ticker.replace(".SA","")
    print(ticker)
    url = URLS[ticker]
    
    driver = webdriver.Firefox()
    driver.get(url)
    
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "#bsPainelLucratividade > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div")
        )
    )
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "#bsPainelLucratividade > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div")
        )
    )

    # Se chegou aqui, elemento existe — executa o código
    res = driver.find_element(By.CSS_SELECTOR, "#bsPainelLucratividade > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text
    ind = driver.find_element(By.CSS_SELECTOR, "#abaIndicadores01").text
    div = driver.find_element(By.CSS_SELECTOR, "#bsPainelDivida > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text
    flx = driver.find_element(By.CSS_SELECTOR, "#bsPainelFluxoCaixa > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text
    bal = driver.find_element(By.CSS_SELECTOR, "#bsBalancoPatrimonial > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text
    
#add label for first column and remove symbols from variables
    resz = "Datas\n"+res.replace(".","").replace(" ","\n").replace("%","").replace(",",".")
    indz = "Datas\n"+ind.replace(".","").replace(" ","\n").replace("%","").replace(",",".")
    divz = "Datas\n"+div.replace(".","").replace(" ","\n").replace("%","").replace(",",".")
    flxz = "Datas\n"+flx.replace(".","").replace(" ","\n").replace("%","").replace(",",".")
    balz = "Datas\n"+bal.replace(".","").replace(" ","\n").replace("%","").replace(",",".")
    reszs = resz.splitlines()
    indzs = indz.splitlines()
    divzs = divz.splitlines()
    flxzs = flxz.splitlines()
    balzs = balz.splitlines()
#define headers for data
#resultados
    reszsr = defaultdict(list)
    key = None
    reset_key = True
    for item in reszs:
        if str(item).isdigit() or item.startswith('-') or item.endswith('%') or item.find(".")>0:
            reszsr[key].append(item)
            reset_key = True
        elif reset_key:
            key = item
            reset_key = False
        else:
            key += f' {item}'
#indicadores
    indzsr = defaultdict(list)
    key = None
    reset_key = True
    for item in indzs:
        if str(item).isdigit() or item.startswith('-') or item.endswith('%') or item.find(".")>0:
            indzsr[key].append(item)
            reset_key = True
        elif reset_key:
            key = item
            reset_key = False
        else:
            key += f' {item}'
#dívidas
    divzsr = defaultdict(list)
    key = None
    reset_key = True
    for item in divzs:
        if str(item).isdigit() or item.startswith('-') or item.endswith('%') or item.find(".")>0:
            divzsr[key].append(item)
            reset_key = True
        elif reset_key:
            key = item
            reset_key = False
        else:
            key += f' {item}'
#fluxo de caixa
    flxzsr = defaultdict(list)
    key = None
    reset_key = True
    for item in flxzs:
        if str(item).isdigit() or item.startswith('-') or item.endswith('%') or item.find(".")>0:
            flxzsr[key].append(item)
            reset_key = True
        elif reset_key:
            key = item
            reset_key = False
        else:
            key += f' {item}'
#balanço patrimonial
    balzsr = defaultdict(list)
    key = None
    reset_key = True
    for item in balzs:
        if str(item).isdigit() or item.startswith('-') or item.endswith('%') or item.find(".")>0:
            balzsr[key].append(item)
            reset_key = True
        elif reset_key:
            key = item
            reset_key = False
        else:
            key += f' {item}'
#creating dataframes
    dfres = pd.DataFrame(reszsr).set_index(["Datas"]).replace('---','0')
    dfdiv = pd.DataFrame(divzsr).set_index(["Datas"]).replace('---','0')
    dfflx = pd.DataFrame(flxzsr).set_index(["Datas"]).replace('---','0')
    dfbal = pd.DataFrame(balzsr).set_index(["Datas"]).replace('---','0')
    dfind = pd.DataFrame(indzsr).set_index(["Datas"]).replace('---','0')  
#dataframes tò float
    dfresf = dfres.astype(float)
    dfdivf = dfdiv.astype(float)
    dfflxf = dfflx.astype(float)
    dfbalf = dfbal.astype(float)
    dfindf = dfind.astype(float)
#rename tabela indicadores
    dfindf.columns = ['LPA','P/L','VPA','P/VP','P/EBIT','P/EBITDA','P/Ativos','EBITDA','Marg EBITDA','PSR','ROE','ROA','ROIC','EV/EBIT','EV/EBITDA','Div Yield','Div Payout','Beta','Valorização','Negócios/dia','Volume Diário']
#creating final dataframe
    ca = dfresf.merge(dfdivf, left_index=True, right_index=True)
    cb = ca.merge(dfflxf, left_index=True, right_index=True)
    cc = cb.merge(dfbalf, left_index=True, right_index=True)
    complete = cc.merge(dfindf, left_index=True, right_index=True)
#save files
    filepath = RAW_DATA_DIR / output_dir / f"{ticker.replace('.SA','')}.csv"
    if not os.path.exists(filepath):
        os.makedirs(filepath)
        complete.to_csv(os.path.join(filepath, f"{ticker}.csv"))
        print("SUCCESS")
        driver.quit()

print("FINISHED")

BBAS3


TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:202:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:555:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [ ]:
import yfinance as yf
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


import sys, os
sys.path.append(os.path.abspath(".."))

from pantheon.config import RAW_DATA_DIR, TICKERS, START_DATE, END_DATE

2025-10-06 18:15:08.637 | INFO     | pantheon.config:<module>:11 - PROJ_ROOT path is: C:\Users\User\Documents\GitHub\pantheon-quant


In [ ]:
from econodata import Empresa
import pandas as pd

# Lista de CNPJs das empresas (exemplo: '00.000.000/0001-91')
cnpjs = ['00.000.000/0001-91', '00.000.000/0001-92']

# Função para coletar dados financeiros trimestrais
def coletar_dados(cnpj):
    empresa = Empresa(cnpj)
    # Obtém os demonstrativos financeiros trimestrais
    dados_trimestrais = empresa.demonstrativos(trimestre=True, ano_inicio=2015)
    # Converte para DataFrame
    df = pd.DataFrame(dados_trimestrais)
    df['CNPJ'] = cnpj
    return df

# Coleta os dados para todos os CNPJs
dados_completos = pd.concat([coletar_dados(cnpj) for cnpj in cnpjs], ignore_index=True)

# Exibe as primeiras linhas dos dados coletados
print(dados_completos.head())


In [17]:
t = yf.Ticker("PETR4.SA")

dfs = {
            "income": t.quarterly_financials.T,
            "balance": t.quarterly_balance_sheet.T,
            "cashflow": t.quarterly_cashflow.T,
        }



In [18]:
combined_df = pd.DataFrame(columns=["Date"])
for name, df in dfs.items():
    df['Date'] = pd.to_datetime(df.index)
    combined_df = pd.merge(combined_df, df, on=["Date"], how="outer")

combined_df

,Date,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,...,Depreciation Amortization Depletion,Depreciation And Amortization,Amortization Cash Flow,Amortization Of Intangibles,Depreciation,Operating Gains Losses,Pension And Employee Benefit Expense,Earnings Losses From Equity Investments,Net Foreign Currency Exchange Gain Loss,Net Income From Continuing Operations
0,2024-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-06-30,-2.981440e+08,0.077000,7.609000e+09,-3.872000e+09,-3.872000e+09,-3.440000e+08,3.138000e+09,8.806000e+09,3.737000e+09,...,3.138000e+09,3.138000e+09,33000000.0,33000000.0,3.105000e+09,8.806000e+09,1.702000e+09,188000000.0,7.040000e+09,-3.250000e+08
2,2024-09-30,1.688612e+07,0.272357,1.196000e+10,6.200000e+07,6.200000e+07,5.870000e+09,2.983000e+09,8.572000e+09,1.202200e+10,...,2.983000e+09,1.220000e+08,33000000.0,33000000.0,8.900000e+07,6.970000e+08,4.090000e+08,23000000.0,1.680000e+08,5.891000e+09
3,2024-12-31,-1.435428e+09,0.221722,6.866000e+09,-6.474000e+09,-6.474000e+09,-2.780000e+09,2.996000e+09,8.046000e+09,3.920000e+08,...,2.996000e+09,5.857000e+09,34000000.0,34000000.0,5.823000e+09,6.938000e+09,3.900000e+08,323000000.0,6.264000e+09,-2.766000e+09
4,2025-03-31,8.768880e+08,0.342000,1.074700e+10,2.564000e+09,2.564000e+09,5.974000e+09,3.247000e+09,7.646000e+09,1.331100e+10,...,3.247000e+09,3.247000e+09,29000000.0,29000000.0,3.218000e+09,-1.677000e+09,4.170000e+08,-82000000.0,-1.955000e+09,5.995000e+09
5,2025-06-30,4.414279e+08,0.257994,9.429000e+09,1.711000e+09,1.711000e+09,4.734000e+09,3.697000e+09,7.545000e+09,1.114000e+10,...,3.697000e+09,3.697000e+09,43000000.0,43000000.0,3.654000e+09,-8.830000e+08,4.300000e+08,-47000000.0,-1.252000e+09,4.757000e+09


In [ ]:
output_dir = "fund_rawdata"
os.makedirs(RAW_DATA_DIR / output_dir, exist_ok=True)


# Função para limpar ticker
def clean_ticker(ticker):
    return ticker.replace(".SA", "").upper()

# Loop por ticker
for ticker in TICKERS:
    try:
        print(f"📥 Downloading data for {ticker}...")

        t = yf.Ticker(ticker)

        dfs = {
            "income": t.quarterly_financials.T,
            "balance": t.quarterly_balance_sheet.T,
            "cashflow": t.quarterly_cashflow.T,
        }

        combined_list = []
        for name, df in dfs.items():
            if df.empty:
                continue

            df = df.copy()
            df.index = pd.to_datetime(df.index)
            df = df.reset_index().melt(id_vars="index", var_name="account", value_name="value")
            df = df.rename(columns={"index": "date"})
            df["statement"] = name
            df["ticker"] = clean_ticker(ticker)
            df["source"] = "Yahoo Finance"

            combined_list.append(df)

        if not combined_list:
            print(f"⚠ Nenhum dado disponível para {ticker}")
            continue

        combined_df = pd.concat(combined_list, ignore_index=True)

    except Exception as e:
        print(f"❌ Failed to download {ticker}: {e}\n")

print("\n🏁 Done! All files have been successfully generated.")